In [1]:
import requests # HTTP 호출 패키지
import json # json 형식의 파일 관련 패키지
import pandas as pd # 데이터 분석 및 연산 패키지

### 1. KOSPI 오늘자 종목 데이터 수집 및 DBMS에 저장하는 코드 작성 및 제출

In [2]:
# headers 선언
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.57 Whale/3.14.133.23 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}

In [3]:
# payload 데이터 설정 (데이터 속성)

p_data = {
    'bld':'dbms/MDC/STAT/standard/MDCSTAT00601',
    'locale': 'ko_KR',
    'tboxindIdx_finder_equidx0_0': '코스피',
    'indIdx': '1',
    'indIdx2': '001',
    'codeNmindIdx_finder_equidx0_0': '코스피',
    'param1indIdx_finder_equidx0_0': '',
    'trdDd': '20220419',
    'money': '3',
    'csvxls_isNo': 'false'
}

In [4]:
# 데이터의 url 설정
# header-general-Request URL 확인

url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

In [5]:
# HTTP 서버에 데이터 요청
# header-general-Request Method 확인

res = requests.post(url, headers=headers, data=p_data)

In [6]:
# 요청한 데이터를 json 형식으로 html_json에 할당

html_json = json.loads(res.content) 

In [7]:
html_json

{'output': [{'ISU_SRT_CD': '005930',
   'ISU_ABBRV': '삼성전자',
   'TDD_CLSPRC': '67,300',
   'FLUC_TP_CD': '1',
   'STR_CMP_PRC': '600',
   'FLUC_RT': '0.90',
   'MKTCAP': '401,766,365,615,000'},
  {'ISU_SRT_CD': '373220',
   'ISU_ABBRV': 'LG에너지솔루션',
   'TDD_CLSPRC': '435,500',
   'FLUC_TP_CD': '1',
   'STR_CMP_PRC': '2,500',
   'FLUC_RT': '0.58',
   'MKTCAP': '101,907,000,000,000'},
  {'ISU_SRT_CD': '000660',
   'ISU_ABBRV': 'SK하이닉스',
   'TDD_CLSPRC': '112,500',
   'FLUC_TP_CD': '1',
   'STR_CMP_PRC': '3,500',
   'FLUC_RT': '3.21',
   'MKTCAP': '81,900,266,062,500'},
  {'ISU_SRT_CD': '207940',
   'ISU_ABBRV': '삼성바이오로직스',
   'TDD_CLSPRC': '825,000',
   'FLUC_TP_CD': '2',
   'STR_CMP_PRC': '-7,000',
   'FLUC_RT': '-0.84',
   'MKTCAP': '54,586,125,000,000'},
  {'ISU_SRT_CD': '035420',
   'ISU_ABBRV': 'NAVER',
   'TDD_CLSPRC': '312,000',
   'FLUC_TP_CD': '1',
   'STR_CMP_PRC': '5,500',
   'FLUC_RT': '1.79',
   'MKTCAP': '51,183,314,520,000'},
  {'ISU_SRT_CD': '035720',
   'ISU_ABBRV': '카카오'

In [8]:
# json 형태의 결과값을 DataFrame 형식으로 df에 할당

df = pd.json_normalize(html_json['output'])
df.head()

,ISU_SRT_CD,ISU_ABBRV,TDD_CLSPRC,FLUC_TP_CD,STR_CMP_PRC,FLUC_RT,MKTCAP
0,005930,삼성전자,"67,300",1,600,0.90,"401,766,365,615,000"
1,373220,LG에너지솔루션,"435,500",1,"2,500",0.58,"101,907,000,000,000"
2,000660,SK하이닉스,"112,500",1,"3,500",3.21,"81,900,266,062,500"
3,207940,삼성바이오로직스,"825,000",2,"-7,000",-0.84,"54,586,125,000,000"
4,035420,NAVER,"312,000",1,"5,500",1.79,"51,183,314,520,000"


In [9]:
# 날짜, 종목 코드, 종목 이름, 지수 종류만 수집하도록 데이터를 변경

html_json = html_json['output']

In [10]:
# 빈 DataFrame을 만들어주고 날짜, 종목 코드, 종목 이름을 추가

name_hs = pd.DataFrame()

if len(html_json) > 0:
    name_h = [] # 빈 list 생성
    for i in range(len(html_json)):
        ISU_SRT_CD = html_json[i]['ISU_SRT_CD'] # 해당 row의 종목 코드 추출
        ISU_ABBRV = html_json[i]['ISU_ABBRV'] # 해당 row의 종목 이름 추출
        # 날짜, 종목 코드, 종목 이름을 name_hs에 추가
        name_h.append(('20220419', ISU_SRT_CD, ISU_ABBRV))
        
    # columns 재설정
    name_h = pd.DataFrame(name_h, columns = ['date', 'stock_code', 'stock_name'])
    name_h['exchange'] = '코스피'
    name_hs = name_hs.append(name_h, ignore_index=True)

else:
    pass

In [11]:
name_h.head()

,date,stock_code,stock_name,exchange
0,20220419,005930,삼성전자,코스피
1,20220419,373220,LG에너지솔루션,코스피
2,20220419,000660,SK하이닉스,코스피
3,20220419,207940,삼성바이오로직스,코스피
4,20220419,035420,NAVER,코스피


---
#### database 연결 및 데이터 저장

In [12]:
import pymysql as pm

In [13]:
# database 연결

conn = pm.connect(host='localhost', port=3306, db='krx_pr',
                 user='root', passwd='abcd123456', autocommit=True)

In [14]:
cursor = conn.cursor()

In [15]:
# krx_pr db에 kospi 테이블 생성
create_kospi_table = '''CREATE TABLE if NOT EXISTS KOSPI(
	DATE VARCHAR(20),
	stock_code VARCHAR(20),
	stock_name VARCHAR(40),
	EXCHANGE VARCHAR(20),
	PRIMARY KEY (stock_code)
)'''

In [16]:
cursor.execute(create_kospi_table) # kospi 테이블 생성

0

In [17]:
# name_h에 저장된 kospi 데이터를 db에 삽입

for i in range(len(name_h)):
    date = name_h.date.values[i]
    stock_code = name_h.stock_code.values[i]
    stock_name = name_h.stock_name.values[i]
    exchange = name_h.exchange.values[i]
    
    sql = f"replace into kospi (date, stock_code, stock_name, exchange)"\
            f"values ('{date}', '{stock_code}', '{stock_name}', '{exchange}')"
    cursor.execute(sql)
    
conn.commit()

In [18]:
# DBMS 저장 확인
sql = 'select * from kospi'
df = pd.read_sql(sql, conn)
df

,DATE,stock_code,stock_name,EXCHANGE
0,20220419,000020,동화약품,코스피
1,20220419,000040,KR모터스,코스피
2,20220419,000050,경방,코스피
3,20220419,000060,메리츠화재,코스피
4,20220419,000070,삼양홀딩스,코스피
...,...,...,...,...
809,20220419,396690,미래에셋글로벌리츠,코스피
810,20220419,400760,NH올원리츠,코스피
811,20220419,402340,SK스퀘어,코스피
812,20220419,404990,신한서부티엔디리츠,코스피


In [19]:
conn.close() # DB 연결 종료